# Equity Explorer - Multi-Stock Comparison Tool

In [1]:
# yfinance for data
import yfinance as yf

# pandas for data handling
import pandas as pd

# matplotlib.pyplot for visualisation
import matplotlib.pyplot as plt

# ipywidgets and display for interactivity
import ipywidgets as widgets
from IPython.display import display, clear_output

# Output area for results
output_area = widgets.Output()

# Let the user choose tickers and starting year
ticker_input = widgets.Text(
    value='AAPL, MSFT, TSLA',
    placeholder='Enter tickers separated by commas',
    description='Tickers:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

year_slider = widgets.IntSlider(
    value=2010,
    min=2000,
    max=2024,
    step=1,
    description='Start Year:',
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description='📊 Run Analysis',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

# Display everything including output space
format_dropdown = widgets.Dropdown(
    options=['CSV', 'xlsx'],
    value='CSV',
    description='Export as:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

export_button = widgets.Button(
    description='💾 Export',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

display(ticker_input, year_slider, run_button, format_dropdown, export_button, output_area)


Text(value='AAPL, MSFT, TSLA', description='Tickers:', layout=Layout(width='70%'), placeholder='Enter tickers …

IntSlider(value=2010, description='Start Year:', max=2024, min=2000, style=SliderStyle(description_width='init…

Button(button_style='success', description='📊 Run Analysis', layout=Layout(width='200px'), style=ButtonStyle()…

Dropdown(description='Export as:', layout=Layout(width='200px'), options=('CSV', 'xlsx'), style=DescriptionSty…

Button(button_style='info', description='💾 Export', layout=Layout(width='200px'), style=ButtonStyle())

Output()

In [3]:
# grab the historical stock prices ("Close") and rebase them starting at 100
# also removes missing values (dropna)

def run_analysis(button=None):
    with output_area:
        clear_output()

        global price_data  # 👈 This lets the export button use the latest version
        tickers = [t.strip().upper() for t in ticker_input.value.split(",")]
        start_date = f"{year_slider.value}-01-01"

        price_data = pd.DataFrame()

        for ticker in tickers:
            stock = yf.Ticker(ticker)
            hist = stock.history(start=start_date)["Close"].dropna()

            if len(hist) < 1000:
                print(f"{ticker}: insufficient data – skipped.")
                continue

            norm = hist / hist.iloc[0] * 100
            price_data[ticker] = norm

        if price_data.empty:
            print("No valid data for selected tickers.")
            return

        n_years = (price_data.index[-1] - price_data.index[0]).days / 365.25

        cagr_text = ""
        for ticker in price_data.columns:
            start_val = price_data[ticker].iloc[0]
            end_val = price_data[ticker].iloc[-1]
            cagr = (end_val / start_val) ** (1 / n_years) - 1
            cagr_text += f"{ticker}: {cagr:.2%}\n"

        percent_data = price_data - 100

        plt.figure(figsize=(10, 5))
        for col in percent_data.columns:
            plt.plot(percent_data.index, percent_data[col], label=col)

        plt.title("Stock Returns Since Start Date")
        plt.xlabel("Date")
        plt.ylabel("Total Return (%)")
        plt.axhline(0, color="gray", linestyle="--", linewidth=1)
        plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0f}%'))

        plt.text(
            x=percent_data.index[int(len(percent_data) * 0.1)],
            y=percent_data.max().max() * 0.85,
            s=cagr_text,
            fontsize=10,
            bbox=dict(facecolor="white", edgecolor="gray", boxstyle="round,pad=0.5")
        )

        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

In [5]:
run_button.on_click(run_analysis)

In [7]:
def export_data(button=None):
    with output_area:
        if 'price_data' not in globals():
            print("❌ No data available. Please run the analysis first.")
            return

        # Remove timezone info from the index
        export_df = price_data.copy()
        export_df.index = export_df.index.tz_localize(None)

        filename_base = f"stock_returns_{year_slider.value}"
        if format_dropdown.value == 'CSV':
            export_df.to_csv(f"{filename_base}.csv")
            print(f"✅ Data exported as {filename_base}.csv")
        else:
            export_df.to_excel(f"{filename_base}.xlsx")
            print(f"✅ Data exported as {filename_base}.xlsx")
# Remove old handlers first (if any)
export_button._click_handlers.callbacks.clear()

# Then attach the function
export_button.on_click(export_data)